In [111]:
!pip install --upgrade pathway bokeh

In [112]:
!pip install pathway bokeh --quiet

In [123]:
import pandas as pd
import pathway as pw
import panel as pn
import bokeh.plotting
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource
import numpy as np

output_notebook()


In [124]:
# Load dataset and merge datetime columns
df = pd.read_csv("dataset.csv")
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"], format="%d-%m-%Y %H:%M:%S")

# Map categorical features to numerical values
df["TrafficLevel"] = df["TrafficConditionNearby"].map({"low": 0, "medium": 1, "high": 2}).fillna(0).astype(int)
df["VehicleTypeWeight"] = df["VehicleType"].map({"car": 1.0, "bike": 0.5, "truck": 1.5}).fillna(0.0) # Fill potential NaN values with 0.0

# Save selected features for Model 2 to a new CSV
df[[
    "Timestamp", "Occupancy", "Capacity", "QueueLength",
    "TrafficLevel", "IsSpecialDay", "VehicleTypeWeight"
]].to_csv("parking_stream_model2.csv", index=False)

In [125]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficLevel: int
    IsSpecialDay: int
    VehicleTypeWeight: float


In [126]:
stream = pw.demo.replay_csv("parking_stream_model2.csv", schema=ParkingSchema, input_rate=100)
stream = stream.with_columns(
    t = stream.Timestamp.dt.strptime("%Y-%m-%d %H:%M:%S")
)


In [127]:
BASE_PRICE = 10.0
LAMBDA = 0.5

ALPHA = 1.5    # Occupancy effect
BETA = 0.8     # Queue length effect
GAMMA = 0.6    # Traffic penalty
DELTA = 1.0    # Special day boost
EPSILON = 0.5  # Vehicle type weight


In [128]:
import pandas as pd

try:
    # Read only the header row of the CSV
    csv_header = pd.read_csv("parking_stream_model2.csv", nrows=0).columns.tolist()
    print("Header of parking_stream_model2.csv:")
    print(csv_header)
except FileNotFoundError:
    print("Error: parking_stream_model2.csv not found.")
except Exception as e:
    print(f"An error occurred while reading the CSV header: {e}")

Header of parking_stream_model2.csv:
['Timestamp', 'Occupancy', 'Capacity', 'QueueLength', 'TrafficLevel', 'IsSpecialDay', 'VehicleTypeWeight']


In [129]:
print(df['TrafficLevel'].dtype)

int64


In [130]:
# Step 6: Compute real-time demand and pricing
model2_intermediate = stream.with_columns(
    t = stream.t,
    queue_length_effect = BETA * stream.QueueLength
)

model2_demand = model2_intermediate.with_columns(
    demand = (
        ALPHA * (model2_intermediate.Occupancy / model2_intermediate.Capacity) +
        model2_intermediate.queue_length_effect -
        GAMMA * model2_intermediate.TrafficLevel +
        DELTA * model2_intermediate.IsSpecialDay +
        EPSILON * model2_intermediate.VehicleTypeWeight
    )
)

model2_normalized_demand = model2_demand.with_columns(
    # Normalize demand by clamping between 0 and 1
    normalized_demand = pw.apply(lambda x: max(0, min(1, x)), model2_demand.demand)
)

model2_result = model2_normalized_demand.with_columns(
    # Calculate price using pw.apply to handle the multiplication and addition with scalars
    price = pw.apply(lambda norm_dem: round(BASE_PRICE * (1 + LAMBDA * norm_dem), 2), model2_normalized_demand.normalized_demand)
)

# Step 7: Define a Bokeh plotting function with subsampled markers
def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Model 2 - Streaming Demand-Based Price",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="green", legend_label="Model 2 Price")

    # Subsample red markers every 100th point
    if "t" in source.data and len(source.data["t"]) > 0:
        indices = list(range(0, len(source.data["t"]), 100))
        subsampled_data = {k: [source.data[k][i] for i in indices] for k in source.data}
        subsampled_source = ColumnDataSource(subsampled_data)
        fig.circle("t", "price", source=subsampled_source, size=6, color="orange")

    return fig

# Step 8: Bind the plot to the live stream
viz = model2_result.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [131]:
# Step 9: Start the stream (in the background)
%%capture --no-display
pw.run()

Output()